In [1]:
import pandas as pd
import re
import os
from time import time
from sys import getsizeof

Pourquoi a-t-on bien traité le fichier ?

- Le nombre d'auteurs et de papiers correspond à la somme des fichiers dans les dossier abstracts/{year}


## Construction d'un dico Publication par auteur

In [6]:
import pandas as pd
import numpy as np
import re
import os
from time import time
from collections import defaultdict


years = sorted(os.listdir('abstracts'))

files = []  # attribu en POO ?
nb_files = 0
i = 0

for year in years:
    files.append(os.listdir(f'abstracts/{year}'))
    nb_files += len(files[i])
    i += 1

print(f'Le fichier abstracts contient {nb_files} fichiers.')

def pre_processing_abstracts():
    dict_p = defaultdict(list)
    dict_a = defaultdict(list)

    for year, file_by_year in zip(years, files):
        for file in sorted(file_by_year):
            with open(f"abstracts/{year}/{file}","r") as f:
                nb_line_author = 0 # for pass when there is two lines with Authors => 9201039.abs
                for line in f:
                    if nb_line_author < 1:
                        if line[:7]=="Paper: ":
                            tmp_paper = line[14:21]

                        if line[:9] == "Authors: ":
                            nb_line_author += 1
                            dict_p[tmp_paper] = re.split(' and |, |& ', line[9:-1])

                        if line[:8] == "Author: ": 
                            nb_line_author +=  1
                            dict_p[tmp_paper] = re.split(' and |, |& ', line[8:-1])

    for paper, authors in dict_p.items():
        for author in authors:
            dict_a[author].append(paper)
    print("Abstracts files' pre-processing is done.")
    return dict_p, dict_a


# Mauvaises entrées:

# '18 pages': ['9201047'] => Dieter Lust and Carlos Munoz
# ligne 9206083 :  ['J.A. Casas', "F. G\\'omez", 'C. Mu\\~noz'] tild

Le fichier abstracts contient 29555 fichiers.


In [8]:
dict_p, dict_a = pre_processing_abstracts()
nb_pub = len(dict_p)
np_author = len(dict_a)
print(f'On recence {nb_pub} publications et {np_author} auteurs dans notre jeu de données.')

Abstracts files' pre-processing is done.
On recence 29555 publications et 19675 auteurs dans notre jeu de données.


In [ ]:
def quote0(author):
    """
    fonction quote avec une profondeur N = 1
    """
    author_quoted = []
    paper_quoted = dict_a[author] # on récupère les contributions de l'auteur
    
    # pour chaque contribution, on regarde les papiers cités
    # on récupère les auteurs de chaque papier
    # et si un auteur n'est pas déjà dans les auteurs cités, on le rajoute à author_quoted
    for paper in paper_quoted: 
        for author in dict_p[paper]:
            if not author in author_quoted:
                author_quoted.append(author)
                
    return author_quoted

print(quote0("C. Itzykson"))

In [ ]:
def quote(author, N=1):
    """
    Chercher pour chaque auteur qui la influencé à son 
    tour mais si auteur dans profondeur n-1 ne pas ajouter.
    Aussi voir la question de la pondération.
    """
    author_quoted = [[author]]
    tmp = []
    for i in range(N):
        for author in author_quoted[-1]:
            tmp.append(quote0(author))
            if i == N-1: print(tmp)
        for i in tmp:
            author_quoted.append([author for author in tmp if author not in author_quoted[-1]])
    return author_quoted

#quote("C. Itzykson", 2)

## Matrice d'adjacence

Essayons de créer un matrice d'adjacence de notre communauté de scientifique.
On a deux dictionnaires. Un avec les publications en clés et les auteurs en valeurs. Un autre avec les auteurs en clés et les papiers en valeurs.
Créons avant tout la matrice!

In [ ]:
print(len(dict_a))

adj_matrix_shape = (len(dict_a),len(dict_a))
adj_matrix = np.zeros(adj_matrix_shape)

Comment créé une ligne ?

In [13]:
def adj_matrix():
    for author in dict_a.keys(): # on regarde pour chaque chaque auteurs ie chaque clé du dictionnaire
        # à l'aide de la fonction quote0 on regarde avec quels auteurs un auteurs à un lien de profondeurs 1
        # (se poser la question de la pondération ?)
        # puis on met un 1 (ou val intensité cf ponderation) à la colone des auteurs récupérer dans quote0
        # ie auteur cité par notre auteur
        n = len(dict_a)
        for author in quote0(author):
            i = -1 # position de la clé ie de notre autreur dans dict_a
            j = -1 # position de l'auteur cité par notre auteur dans dict_a
            adj_matrix[i][j] = 1
        return adj_matrix_matrix

In [14]:
# faire une matrice d'adjacence avec un df
# index => noms auteurs
# columns => noms auteurs
# cases => intensité de lien entre les auteurs (orienté)